<a href="https://colab.research.google.com/github/jialun1221/scRNA-seq/blob/main/Preprocessing_step1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocessing and clustering Cells
### Part 1. Data selection

In May 2017, this started out as a demonstration that Scanpy would allow to reproduce most of Seurat's [guided clustering tutorial](http://satijalab.org/seurat/pbmc3k_tutorial.html) ([Satija et al., 2015](https://doi.org/10.1038/nbt.3192)).

We gratefully acknowledge Seurat's authors for the tutorial! In the meanwhile, we have added and removed a few pieces.

The data consist of *3k PBMCs from a Healthy Donor* and are freely available from 10x Genomics ([here](http://cf.10xgenomics.com/samples/cell-exp/1.1.0/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz) from this [webpage](https://support.10xgenomics.com/single-cell-gene-expression/datasets/1.1.0/pbmc3k)). On a unix system, you can uncomment and run the following to download and unpack the data. The last line creates a directory for writing processed data.

In this notebook, we will compute ***Data Selection***. We will ***drop the cells that cotain Lewy Body Dementia***, and create a new AnnData object that contains only PD and control cells. All other features of the original AnnData will reamin.

In [ ]:
!pip install scanpy
import numpy as np
import pandas as pd
import scanpy as sc

In [ ]:
!mkdir data
!mkdir write

In [ ]:
!pip install matplotlib==3.1.3
from numpy import inf

In [ ]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

scanpy==1.9.5 anndata==0.9.2 umap==0.5.4 numpy==1.23.5 scipy==1.11.2 pandas==1.5.3 scikit-learn==1.2.2 statsmodels==0.14.0 pynndescent==0.5.10


In [ ]:
#file to store new Anndata object
new_anndata = 'write/new_anndata_all.h5ad'

Read in the count matrix into an [AnnData](https://anndata.readthedocs.io/en/latest/anndata.AnnData.html) object, which holds many slots for annotations and different representations of the data. It also comes with its own HDF5-based file format: `.h5ad`.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
adata = sc.read_h5ad("drive/MyDrive/scRNA ML classifier/data_objects_May_2022/raw cell data files/All_PD_ann.h5ad")

In [ ]:
adata

AnnData object with n_obs × n_vars = 434340 × 41625
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'Cell_Subtype', 'Cell_Type', 'disease__ontology_label', 'organ__ontology_label'
    var: 'features'

####**Checking (optional to run)**
    
Start with some basic checking.

In [ ]:
adata.var_names #this gives genes!

Index(['RP11-34P13.3', 'FAM138A', 'OR4F5', 'RP11-34P13.7', 'RP11-34P13.8',
       'RP11-34P13.14', 'RP11-34P13.9', 'FO538757.3', 'FO538757.2',
       'AP006222.2',
       ...
       'RNU2-71P', 'RNA5SP252', 'AC116533.2', 'AC114494.1', 'RN7SL424P',
       'RN7SL739P', 'MIR4502', 'RNU4-10P', 'RN7SL865P', 'RNU6-191P'],
      dtype='object', length=41625)

In [ ]:
adata.obs_names #this are labels

Index(['pPDCN4340DAPIA030419_AATGCCACACAAGCAG-1',
       'pPDCN4340DAPIA030419_AGACTCAGTCACAATC-1',
       'pPDCN4340DAPIA030419_ATACTTCCAGCGTTGC-1',
       'pPDCN4340DAPIA030419_ATATCCTGTGTGTTTG-1',
       'pPDCN4340DAPIA030419_ATCCACCGTGGGTTGA-1',
       'pPDCN4340DAPIA030419_ATCCGTCGTCACGACC-1',
       'pPDCN4340DAPIA030419_ATCGTAGCAAGCTGCC-1',
       'pPDCN4340DAPIA030419_ATGCCTCTCTGGGATT-1',
       'pPDCN4340DAPIA030419_ATTCCATAGTTTCTTC-1',
       'pPDCN4340DAPIA030419_CACACAACACTATCGA-1',
       ...
       'pPDsHSrSNxi3298d200429PosB_TCGTGGGAGATCCCAT-1',
       'pPDsHSrSNxi3298d200429PosB_TGCTCCAGTGGCTCTG-1',
       'pPDsHSrSNxi3298d200429PosB_ATGGGTTTCTATCCAT-1',
       'pPDsHSrSNxi3298d200429PosB_AAGCCATAGGATACCG-1',
       'pPDsHSrSNxi3298d200429PosB_ATTCTTGTCCGCAAAT-1',
       'pPDsHSrSNxi3298d200429PosB_CATACTTGTCTGATCA-1',
       'pPDsHSrSNxi3298d200429PosB_TTTGTTGTCCTTGAAG-1',
       'pPDsHSrSNxi3298d200429PosB_ACACTGACACTAGAGG-1',
       'pPDsHSrSNxi3298d200429PosB_CAACAA

In [ ]:
adata.obs_names = [f"Cell_{i:d}" for i in range(adata.n_obs)]
adata.var_names = [f"Gene_{i:d}" for i in range(adata.n_vars)]
print(adata.obs_names[:10])
print(adata.var_names[:10])

Index(['Cell_0', 'Cell_1', 'Cell_2', 'Cell_3', 'Cell_4', 'Cell_5', 'Cell_6',
       'Cell_7', 'Cell_8', 'Cell_9'],
      dtype='object')
Index(['Gene_0', 'Gene_1', 'Gene_2', 'Gene_3', 'Gene_4', 'Gene_5', 'Gene_6',
       'Gene_7', 'Gene_8', 'Gene_9'],
      dtype='object')


In [ ]:
#Check how many rows are unwanted data.
adata.obs.loc[adata.obs['disease__ontology_label'].str.contains("Lewy body dementia", case=False)]

,orig.ident,nCount_RNA,nFeature_RNA,Cell_Subtype,Cell_Type,disease__ontology_label,organ__ontology_label
Cell_141134,86,18376.0,6077,Astro_GBP2_SPOCD1,astro,Lewy body dementia,substantia nigra pars compacta
Cell_141135,86,10638.0,4211,Astro_GJB6_OXTR,astro,Lewy body dementia,substantia nigra pars compacta
Cell_141136,86,10009.0,4048,Astro_GJB6_OXTR,astro,Lewy body dementia,substantia nigra pars compacta
Cell_141137,86,9735.0,3777,Astro_GJB6_OXTR,astro,Lewy body dementia,substantia nigra pars compacta
Cell_141138,86,8617.0,3824,Astro_GJB6_OXTR,astro,Lewy body dementia,substantia nigra pars compacta
...,...,...,...,...,...,...,...
Cell_430462,30,3193.0,1874,SOX6_DDT,da,Lewy body dementia,substantia nigra pars compacta
Cell_430463,30,2991.0,1678,SOX6_DDT,da,Lewy body dementia,substantia nigra pars compacta
Cell_430464,30,2463.0,1531,SOX6_DDT,da,Lewy body dementia,substantia nigra pars compacta
Cell_430465,30,999.0,804,CALB1_RBP4,da,Lewy body dementia,substantia nigra pars compacta


###**Data selection**

Drop the Lewy body dementia:

In [ ]:
adata.obs = adata.obs.reset_index() #Set index for the labels
k = adata.obs #create a variable for further uses (a DataFrame)

In [ ]:
y = k.index[k['disease__ontology_label'] == 'Lewy body dementia'].tolist() #get the index that contains the Lewy Body Dementia samples, stored in variable y (a list)

In [ ]:
m = adata.X.toarray() #convert sparse matrix X to array

Conduct data selection separately in adata.X and adata.obs.

In [ ]:
m = np.delete(m, obj = y, axis=0) #delete rows that contain Lewy Body Dementia according to the previously generated index stored in y

In [ ]:
#drop command for adata.obs
adata.obs.drop(adata.obs.index[adata.obs['disease__ontology_label'] == 'Lewy body dementia'], inplace=True)
adata.obs

###**making new AnnData object**

In [ ]:
pip install anndata

In [ ]:
#Command for making a new AnnData object. For each parameter, need to make a deep copy of the original object.
new = sc.AnnData(X = m,
  obs = adata.obs.copy(),
  var = adata.var.copy(),
  uns = adata.uns.copy(),
  obsm = adata.obsm.copy(),
  varm = adata.varm.copy(),
  layers = adata.layers.copy(),
  raw = adata.raw.copy(),
  dtype = "float32",
  shape = None,
  #filename = NULL,
  #filemode = NULL,
  obsp = adata.obsp.copy(),
  varp = adata.varp
  )
#varp = adata.varp.copy() would give me error but direct assignment would not

In [ ]:
#A random line that I found necessary for the object to work.
new.__dict__['_raw'].__dict__['_var'] = adata.__dict__['_raw'].__dict__['_var'].rename(columns={'_index': 'features'})

In [ ]:
new.write(new_anndata)

In [ ]:
print(adata.X.shape, new.X.shape) #Now the new AnnData object is generated. Check the dimension!

A new AnnData object is created, and stored in the Colab disk. Navigate to the folder button on the left side panel, and click on "write", you will find the `new_anndata.h5ad file` here. Please either download it to your local disk, then upload to your google drive; or move to your drive folder by dragging it to the `drive` folder.

---
The purpose of creating a new AnnData is to keep the accessibility of other features, stored in `adata.obsm`, `adata.varm`, etc.
